##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：REST Function 呼叫設定

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/rest/Function_calling_config_REST.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


指定 `function_calling_config` 可讓你控制當指定 `tools` 時，Gemini API 如何作用。例如，你可以選擇僅允許自由文字輸出 (停用函式呼叫)、強制它從 `tools` 中提供的函式子集選擇，或讓它自動作用。

本指南假設你已熟悉函式呼叫。如需入門，請查看 [REST 函式呼叫](./Function_calling_REST.ipynb) 食譜。


此筆記本提供快速的程式碼範例，讓你了解如何使用 `curl` 開始函式呼叫。

你可以在 Google Colab 中執行此操作，或將 `curl` 指令複製/貼到終端機中。

若要執行此筆記本，你的 API 金鑰必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 機密中。如果你在其他環境中執行，則可以將金鑰儲存在環境變數中。請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 以瞭解更多資訊。


In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## 使用工具設定模型

這個範例提供模型一些控制假設照明系統的功能。要使用這些功能需要按特定順序呼叫它們。例如，你必須先開啟燈光系統才能變換顏色。

雖然你可以將這些功能直接傳遞給模型並讓它嘗試正確呼叫，但指定 `function_calling_config` 能讓你精確控制模型有哪些可用功能。

將工具寫入 `tools.json` 中，以便你在後續步驟中參考它。


In [ ]:
%%file tools.json
{
  "function_declarations": [
    {
      "name": "enable_lights",
      "description": "Turn on the lighting system.",
      "parameters": { "type": "object" }
    },
    {
      "name": "set_light_color",
      "description": "Set the light color. Lights must be enabled for this to work.",
      "parameters": {
        "type": "object",
        "properties": {
          "rgb_hex": {
            "type": "string",
            "description": "The light color as a 6-digit hex string, e.g. ff0000 for red."
          }
        },
        "required": [
          "rgb_hex"
        ]
      }
    },
    {
      "name": "stop_lights",
      "description": "Turn off the lighting system.",
      "parameters": { "type": "object" }
    }
  ]
}

Overwriting tools.json


## 純文字模式：`NONE`

如果你已經提供工具給模型，但不想在目前的對話回合中使用這些工具，請將模式指定為 `NONE`。`NONE` 會指示模型不要進行任何函式呼叫，而會將其視為未提供任何函式。


In [ ]:
%%bash
curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d @<(echo '
  {
    "system_instruction": {
      "parts": {
        "text": "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."
      }
    },
    "tools": [' $(cat tools.json) '],

    "tool_config": {
      "function_calling_config": {"mode": "none"}
    },

    "contents": {
      "role": "user",
      "parts": {
        "text": "What can you do?"
      }
    }
  }
') 2>/dev/null |sed -n '/"content"/,/"finishReason"/p'

      "content": {
        "parts": [
          {
            "text": "As your lighting system, I can turn the lights on and off, and I can set the color of the lights. \n"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",


## 自動模式：`AUTO`

即使允許模型決定要以文字回應或呼叫特定函式，你也可以將模式指定為 `AUTO`。


In [ ]:
%%bash
curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d @<(echo '
  {
    "system_instruction": {
      "parts": {
        "text": "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."
      }
    },
    "tools": [' $(cat tools.json) '],

    "tool_config": {
      "function_calling_config": {"mode": "auto"}
    },

    "contents": {
      "role": "user",
      "parts": {
        "text": "Light this place up!"
      }
    }
  }
') 2>/dev/null |sed -n '/"content"/,/"finishReason"/p'

      "content": {
        "parts": [
          {
            "functionCall": {
              "name": "enable_lights",
              "args": {}
            }
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",


## 函式呼叫模式：`ANY`

將模式設為 `ANY` 將強制模型進行函式呼叫。藉由設定 `allowed_function_names`，模型將僅從這些函式中進行選擇。如果未設定，`tools` 內的所有函式都將是函式呼叫的候選對象。

在這個範例系統中，如果燈光已經開啟，使用者就可以變更顏色或將燈光關閉。


In [ ]:
%%bash
curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d @<(echo '
  {
    "system_instruction": {
      "parts": {
        "text": "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."
      }
    },
    "tools": [' $(cat tools.json) '],

    "tool_config": {
      "function_calling_config": {
        "mode": "any",
        "allowed_function_names": ["set_light_color", "stop_lights"]
      }
    },

    "contents": {
      "role": "user",
      "parts": {
        "text": "Make this place PURPLE!"
      }
    }
  }
') 2>/dev/null |sed -n '/"content"/,/"finishReason"/p'

      "content": {
        "parts": [
          {
            "functionCall": {
              "name": "set_light_color",
              "args": {
                "rgb_hex": "9400d3"
              }
            }
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",


## 進一步閱讀

查看 [函式呼叫食譜](./Function_calling_REST.ipynb) 以進一步瞭解函式呼叫。
